### General advice (delete this cell before submitting for review)
> - When choosing a location for your analysis, **select an area that has data on both the `NCI` and `DEA Sandbox`** to allow your code to be run on both environments. 
For example, you can check this for Landsat using the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all products).
As of September 2019, the `DEA Sandbox` has a single year of continental Landsat data for 2015-16, and the full 1987-onward time-series for three locations (Perth WA, Brisbane QLD, and western NSW).
> - When adding **Products used**, embed the hyperlink to that specific product on the DEA Explorer using the `[text link](product url)` syntax.
> - When writing in Markdown cells, start each sentence on a **new line**.
This makes it easy to see changes through git commits.
> - Use Australian English in markdown cells and code comments.
> - Use the [PEP8 standard](https://www.python.org/dev/peps/pep-0008/) for code. To make sure all code in the notebook is consistent, you can use the `jupyterlab_code_formatter` tool: select each code cell, then click `Edit` and then one of the `Apply X Formatter` options (`YAPF` or `Black` are recommended). This will reformat the code in the cell to a consistent style.
> - For additional guidance, refer to the style conventions and layouts in approved `develop` branch notebooks. 
Examples include
    - [Frequently_used_code/Using_load_ard.ipynb](./Frequently_used_code/Using_load_ard.ipynb)
    - [Real_world_examples/Coastal_erosion.ipynb](./Real_world_examples/Coastal_erosion.ipynb)
    - [Scripts/dea_datahandling.py](./Scripts/dea_datahandling.py)
> - In the final notebook cell, include a set of relevant tags which are used to build the DEA User Guide's [Tag Index](https://docs.dea.ga.gov.au/genindex.html). 
Use all lower-case (unless the tag is an acronym), separate words with spaces (unless it is the name of an imported module), and [re-use existing tags](https://github.com/GeoscienceAustralia/dea-notebooks/wiki/List-of-tags).
Ensure the tags cell below is in `Raw` format, rather than `Markdown` or `Code`.


# Open and run analysis on multiple polygons
* **Compatability:** Notebook currently compatible with the `NCI`|`DEA Sandbox` environment only
* **Products used:** 
[`s2a_ard_granule`](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[`s2b_ard_granule`](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule),
[`ga_ls5t_ard_3`](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[`ga_ls7e_ard_3`](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[`ga_ls8c_ard_3`](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)

### Background
Many users need to run analyses on their own areas of interest. A common use case involves running the same analysis across multiple polygons in a shapefile.

### Description
If we have a vector file containing multiple polygons, we can use the python package [fiona](https://fiona.readthedocs.io/en/latest/manual.html#reading-vector-data) to open it and iterate through the files, retrieving and plotting data from Digital Earth Australia.
1. First we open the polygon using the [fiona](https://fiona.readthedocs.io/en/latest/manual.html#reading-vector-data) package
2. Then we do this
3. Finally we do this


## Getting started
Provide any particular instructions that the user might need, e.g. To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Please note the use of `datacube utilities` package `geometry`: this is important for saving the coordinate reference system of the incoming shapefile in a format that the Digital Earth Australia query can understand. It has the same name as the `shapely.geometry` package, and should be imported later as in the cell below.

In [23]:
%matplotlib inline

import datacube
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
from shapely import geometry

import sys
import xarray as xr

from datacube.storage import masking
from datacube.utils import geometry

sys.path.append('../../Scripts') #probably need to change this after you move the code
sys.path.append('../Scripts')

from dea_datahandling import load_ard
from dea_bandindices import calculate_indices
from dea_plotting import rgb

### Connect to the datacube
Give your datacube app a unique name. 
Ideally, this will be the same as the notebook file name.

In [10]:
dc = datacube.Datacube(app='Analyse_multiple_polygons')

### Analysis parameters

An *optional* section to inform the user of any parameters they'll need to configure to run the notebook:
* `param_name_1`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.
* `param_name_2`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.


In [11]:
# param_name_1 = 'example_value'
# param_name_2 = 'example_value'

## Look at the structure of the vector file
Import the file and take a look at how the file is structured so we understand what we are iterating through. 

### Import our vector file containing multiple polygons
Set the path to the location of the file

In [12]:
# This is the path location of our polygon file. You can change this to another file.
multiple_polys_file = '../../Supplementary_data/Analyse_multiple_polygons/multiple_polys.shp'

In [13]:
#print the file where we are accessing our polygons from
multiple_polys_file

'../../Supplementary_data/Analyse_multiple_polygons/multiple_polys.shp'

Use the python package `fiona` to open and iterate through the polygon file. Working with vector files it's important to save and preserve the Coordinate Reference System, the CRS, so we save this from the `well known text`.

In [14]:
#use the fiona package to open the vector file of the polygons
with fiona.open(multiple_polys_file, 'r') as source:
    #explicity set the coordinate reference system from the source file
    crs=source.crs_wkt
    #iterate through the polygon file and have a look at the dictionaries that we've got in the {'key': 'value'} format.  
    for record in source:
        print(record)
    #then look at the CRS of the file    
    print(crs)

{'type': 'Feature', 'id': '0', 'properties': OrderedDict([('id', 3)]), 'geometry': {'type': 'Polygon', 'coordinates': [[(973435.8333156732, -3554355.769048074), (976356.1110555927, -3554571.451142964), (976219.7217355696, -3557085.685764458), (973232.5083624359, -3556840.9050310687), (973435.8333156732, -3554355.769048074)]]}}
{'type': 'Feature', 'id': '1', 'properties': OrderedDict([('id', 2)]), 'geometry': {'type': 'Polygon', 'coordinates': [[(980959.7463502702, -3560845.1440404137), (983880.0240901897, -3561060.826135304), (983743.6347701667, -3563575.060756798), (980756.4213970329, -3563330.2800234086), (980959.7463502702, -3560845.1440404137)]]}}
{'type': 'Feature', 'id': '2', 'properties': OrderedDict([('id', 1)]), 'geometry': {'type': 'Polygon', 'coordinates': [[(974705.4936402615, -3565359.491861172), (977625.771380181, -3565575.173956062), (977489.382060158, -3568089.4085775563), (974502.1686870242, -3567844.627844167), (974705.4936402615, -3565359.491861172)]]}}
PROJCS["GDA94

Look at a single polygon using its feature id

In [15]:
#choose a single polygon to look at here: to change this change the number below
poly_of_interest=0

In [16]:
#open the file, choose the feature id of the polygon, then look at just that one.
with fiona.open(multiple_polys_file) as shapes:    
    #get crs object from polygon file using datacube.
    crs = geometry.CRS(shapes.crs_wkt)
    for feature in shapes:
        if feature['id'] == str(poly_of_interest):
            #get feature geometry
            feature_geometry=feature['geometry']
            #create datacube geometry.Geometry object
            geom = geometry.Geometry(feature_geometry, crs=crs)
            print(feature['id'])
            print(geom)

0
Geometry({'type': 'Polygon', 'coordinates': [[(973435.8333156732, -3554355.769048074), (976356.1110555927, -3554571.451142964), (976219.7217355696, -3557085.685764458), (973232.5083624359, -3556840.9050310687), (973435.8333156732, -3554355.769048074)]]}, CRS('PROJCS["GDA94 / Australian Albers",GEOGCS["GDA94",DATUM["Geocentric_Datum_of_Australia_1994",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6283"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4283"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",132],PARAMETER["standard_parallel_1",-18],PARAMETER["standard_parallel_2",-36],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3577"]]'))


In [17]:
with fiona.open(multiple_polys_file) as shapes:    
    #get crs object from polygon file using datacube.
    crs = geometry.CRS(shapes.crs_wkt)
    for feature in shapes:
        #get feature geometry
        feature_geometry=feature['geometry']
        #create datacube geometry.Geometry object
        geom = geometry.Geometry(feature_geometry, crs=crs)

In [18]:
def time_buffer(chosen_date, buffer='30 days'):

    """
    Create a buffer of days around a time query. 
    Output is a string the correct format for a datacube query.

    Parameters
        ----------
        chosen_date : str, yyyy-mm-dd
        buffer : str, optional, default is 30 days, needs to be a number of days
        
            
    Returns
    -------
    early_buffer, late_buffer : str
        a tuple of strings to pass to the datacube query function
        e.g. ('2017-12-02', '2018-01-31') for input chosen_date ='2018-01-01' and buffer = '30 days'  
    """
    #use assertions to check we have the correct function input
    assert isinstance(chosen_date, str), "chosen date must be a string in quotes in 'yyyy-mm-dd' format"
    assert isinstance(buffer, str), "buffer must be a string in days e.g. '5 days'"
    
    buffer = pd.Timedelta(buffer)
    chosen_date =pd.to_datetime(chosen_date)
    early_buffer = chosen_date - buffer
    late_buffer = chosen_date + buffer
    #convert back to string
    early_buffer = str(early_buffer)[:10]
    late_buffer = str(late_buffer)[:10]
    return(early_buffer, late_buffer)

Choose a time of interest to look at your polygon

In [28]:
time_of_interest = '2019-03-01'

Then set a buffer around the time to get some data within

In [29]:
early_buffer, late_buffer = time_buffer(time_of_interest, buffer='60 days')

## Demonstrate loading and analysing data for multiple polygons in a loop
Load our query for each polygon, demonstrate calculating an index using dea_bandindices (NDVI) and plotting results 

In [30]:
ndvi.NDVI

<xarray.DataArray 'NDVI' (time: 1, y: 110, x: 126)>
array([[[ 0.17875212,  0.16500734,  0.15951869, ..., -0.57674414,
         -0.5854375 , -0.5761124 ],
        [ 0.16311818,  0.16500734,  0.15951869, ..., -0.5813954 ,
         -0.5813954 , -0.5761124 ],
        [ 0.16311818,  0.15751754,  0.1595941 , ..., -0.5813954 ,
         -0.5813954 , -0.5803624 ],
        ...,
        [ 0.2053735 ,  0.23016733,  0.22739728, ...,  0.46319017,
          0.46319017,  0.4820219 ],
        [ 0.18421051,  0.18421051,  0.15629165, ...,  0.43334234,
          0.43334234,  0.57621044],
        [ 0.21815152,  0.18421051,  0.15629165, ...,  0.3354444 ,
          0.25565428,  0.25565428]]], dtype=float32)
Coordinates:
  * x        (x) float64 9.808e+05 9.808e+05 9.808e+05 ... 9.839e+05 9.839e+05
  * y        (y) float64 -3.561e+06 -3.561e+06 ... -3.564e+06 -3.564e+06
  * time     (time) datetime64[ns] 2018-01-14T00:20:27.714700

In [ ]:
# Load available data from all three Landsat satellites
with fiona.open(multiple_polys_file) as shapes:    
    #get crs object from polygon file using datacube.
    crs = geometry.CRS(shapes.crs_wkt)
    for feature in shapes:
        #get feature geometry
        feature_geometry=feature['geometry']
        #create datacube geometry.Geometry object
        geom = geometry.Geometry(feature_geometry, crs=crs)
        
        #this sets up our datacube query based off the corresponding polygon and the chosen date, the bands we wish to use . 
        query = {'geopolygon': geom,
         'time': (early_buffer,late_buffer),
         'measurements': ['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_nir'],
         'resolution':(-25,25)
        }
        #create a dataset to load our data into for the polygon of i
        landsat_ds = load_ard(dc=dc, 
                              products=['ga_ls5t_ard_3', 
                                        'ga_ls7e_ard_3', 
                                        'ga_ls8c_ard_3'],
                              min_gooddata=0.99, #only take uncloudy scenes
                              ls7_slc_off = False, #ignore stripy data for demonstration    
                              output_crs='EPSG:3577',
                              align=(15, 15),
                              group_by='solar_day',
                              dask_chunks={},
                              **query)
        
        #plot each polygon dataset in rgb
        rgb(landsat_ds, col='time')
        
        #calculate NDVI for each polygon dataset
        ndvi = calculate_indices(landsat_ds, index='NDVI', collection='ga_ls_3')
        
        #plot NDVI for each polygon for the time query
        ndvi.NDVI.plot(col='time', cmap='YlGn', vmin=0, vmax=1)
        plt.show()
        #delete data to manage memory. You may prefer to write out your data and graphs to file - please check out the other Frequently_used_code notebooks for more info.
       # del landsat_ds, ndvi

Loading ga_ls5t_ard_3 data


## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** April 2020

**Compatible `datacube` version:** 

In [14]:
print(datacube.__version__)

1.7+262.g1cf3cea8


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)